# 1. Tải thư viện

In [ ]:
!pip install -r requirements.txt

# 2. Tiền xử lý dữ liệu

In [ ]:
!python ./preprocessDataset.py

# 3. Huấn luyện mô hình

In [ ]:
!CUDA_VISIBLE_DEVICES=0 python "run_summarization.py" \
 --model_name_or_path bart-base   \
 --do_train --do_eval     \
 --train_file data/train.json \
 --validation_file data/valid.json \
 --test_file data/test.json     \
 --output_dir model_checkpoints   --overwrite_output_dir     \
 --max_target_length 4096 \
 --per_device_train_batch_size=2     \
 --per_device_eval_batch_size=4 \
 --text_column text    \
 --summary_column summary \
 --evaluation_strategy steps \
 --eval_accumulation_steps 1000 \
 --save_steps 1000 \
 --eval_steps 1000 \
 --num_train_epochs 20 \
 --fp16

# 4. Đánh giá văn bản 

## 4.1 Quá trình sinh câu dự đoán cho việc tóm tắt văn bản

In [ ]:
!CUDA_VISIBLE_DEVICES=0 python3 run_summarization.py    \
 --model_name_or_path model_checkpoints/checkpoint-20000   \
 --do_predict    \
 --train_file data/train.json \
 --validation_file data/valid.json \
 --test_file data/test.json     \
 --output_dir evaluation     --overwrite_output_dir     \
 --per_device_eval_batch_size=20 \
 --text_column text    \
 --summary_column summary \
 --predict_with_generate

## 4.2 Điểm Bleu và Rouge của kết quả dự đoán của mô hình

In [ ]:
import json
import evaluate

Predicts = []
References = []

with open("data/test.json", "r") as file:
    data = json.load(file)
    References = [item["summary"] for item in data]
with open("evaluation/generated_predictions.txt", "r") as file:
    for line in file:
      Predicts.append(line.strip())

bleu = evaluate.load("bleu")
rouge = evaluate.load('rouge')
results_bleu = bleu.compute(predictions = Predicts, references = References)
results_rouge = rouge.compute(predictions = Predicts, references = References)
print("Bleu score : ", results_bleu)
print("Rouge score: ", results_rouge)